In [75]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import scipy.integrate as integrate
import yaml
import pandas as pd
from scipy.optimize import curve_fit # for fitting the density with a function

## Constants

In [76]:
MEarth = 5.972e24 #kg
R_Earth = 6371 #km
GC = 6.67384e-11 #m3/kg/s2
year = 365.25*3600*24 #s

## Rocky planet class

Defines physical parameters and evolution of a given planet

In [77]:
class Rocky_Planet():

    def __init__(self):
        self.parameters()

    def parameters(self):
        pass 
    
    def read_parameters(self, file):
        with open(file, 'r') as stream:
            try:
                dict_param = yaml.safe_load(stream)
            except yaml.YAMLError as exc:
                print(exc)
        for k, v in dict_param.items():
            setattr(self, k, float(v))
    
    def evolution(self):
        evolution = Evolution(self)
        evolution.energies()
        evolution.profiles()

    def density_Labrosse2015(r, *args):
        """ Equation (5) in Labrosse 2015 """
        rho_0, L_rho, A_rho = args
        return rho_0*(1-r**2/L_rho**2-A_rho*r**4/L_rho**4)

## Evolution class

Calculates the thermal evolution of a given planet

In [88]:
class Evolution():

    def __init__(self, planet):
        self.planet = planet
        
    def calc_L_T(self):
        """ Temperature change length scale """
        return np.sqrt(3.*self.planet.CP/(2.*np.pi*self.planet.alpha_c*self.planet.rho_c*self.planets.GC))
        
    def calc_L_rho(self):
        """ Density change length scale """
        return np.sqrt(3.*self.planet.K_c/(2.*np.pi*G*self.planet.rho_0*self.planet.rho_c)*(np.log(self.planet.rho_c/self.planet.rho_0)+1.))

    def dTL_dr_IC(self, r):
        ''' Melting temperature jump at ICB '''
        return -self.planet.K_c * 2.*self.planet.dTL_dP * r / self.planet.L_rho**2. \
            + 3. * self.planet.dTL_dchi * self.planet.chi0 * r**2. / (self.planet.L_rho**3. * self.fC(self.planet.r_OC / self.planet.L_rho, 0.))

    def fC(self, r, delta): 
        '''fC (Eq. A1 Labrosse 2015)'''
        return r**3. * (1 - 3. / 5. * (delta + 1) * r**2.- 3. / 14. * (delta + 1) \
            * (2 * self.planet.A_rho - delta) * r**4.)

    def fX(self, r, r_IC):
        '''fX (Eq. A15 Labrosse 2015)'''
        return (r)**3. * (-r_IC**2. / 3. / self.planet.L_rho**2. + 1./5. * (1.+r_IC**2./self.planet.L_rho**2.) \
                *(r)**2.-13./70. * (r)**4.) 

    def rho(self, r):
        ''' Density (Eq. 5 Labrosse 2015)'''
        return self.planet.rho_c * (1. - r**2. / self.planet.L_rho**2. - self.planet.A_rho * r**4. / self.planet.L_rho**4.)

    def T_melt(self, r):
        ''' Melting temperature at ICB (Eq. 14 Labrosse 2015)'''
        return self.planet.TL0 - self.planet.K_c * self.planet.dTL_dP * r**2. / self.planet.L_rho**2. + self.planet.dTL_dchi * self.planet.chi0 * r**3. \
                / (self.planet.L_rho**3. * self.fC(self.planet.r_OC / self.planet.L_rho, 0.))
 
    def PL(self, r):
        '''Latent heat power'''
        PL = 4. * np.pi * r**2. * self.T_melt(r) * self.rho(r) * self.planet.DeltaS
        return PL

#     def LH(self, r):
#         LH, i = integrate.quad(self.PL, 0, r)
#         return LH

    def Pc(self, r):
        '''Secular cooling power (Eq. A8 Labrosse 2015)'''
        Pc =  -4. * np.pi / 3. * self.planet.rho_c * self.planet.CP * self.planet.L_rho**3. *\
                (1 - r**2. / self.planet.L_rho**2 - self.planet.A_rho* r**4. / self.planet.L_rho**4.)**(-self.planet.gamma) \
                * (self.dTL_dr_IC(r) + 2. * self.planet.gamma \
                * self.T_melt(r) * r / self.planet.L_rho**2. *(1 + 2. * self.planet.A_rho * r**2. / self.planet.L_rho**2.) \
                /(1 - r**2. / self.planet.L_rho**2. - self.planet.A_rho * r**4. / self.planet.L_rho**4.)) \
                * (self.fC(self.planet.r_OC / self.planet.L_rho, self.planet.gamma))
        return Pc

#     def SC(self, r):
#         SC, i = integrate.quad(self.Pc, 0, r)  
#         return SC

    def Px(self, r):
        ''' Gravitational heat power (Eq. A14 Labrosse 2015)'''
        Px =  8 * np.pi**2 * self.planet.chi0 * self.planet.GC * self.planet.rho_c**2 * self.planet.beta * r**2. \
        * self.planet.L_rho**2. / self.fC(self.planet.r_OC / self.planet.L_rho, 0) \
        * (self.fX(self.planet.r_OC / self.planet.L_rho, r) - self.fX(r / self.planet.L_rho, r))
        return Px

#     def GH(self, r):
#         GH, i = integrate.quad(self.Px, 0, r)
#         return GH
    


    def energies(self):
        
        ''' Latent heat '''
        self.L = 4. * np.pi / 3. * self.planet.rho_c * self.planet.TL0 * self.planet.DeltaS * self.planet.r_IC**3. * (1 - 3. / 5. \
            * (1 + self.planet.K_c / self.planet.TL0 * self.planet.dTL_dP) * self.planet.r_IC**2. / self.planet.L_rho**2. \
            + self.planet.chi0 / (2 * self.fC(self.planet.r_OC / self.planet.L_rho, 0.) * self.planet.TL0) * self.planet.dTL_dchi * self.planet.r_IC**3. / self.planet.L_rho**3.)
        print("Latent heat", self.L,"J")
        
        ''' Secular cooling '''
        self.C = 4. * np.pi / 3. * self.planet.rho_c * self.planet.CP * self.planet.L_rho * self.planet.r_IC**2 * self.fC(self.planet.r_OC / self.planet.L_rho, self.planet.gamma)\
                * (self.planet.dTL_dP * self.planet.K_c - self.planet.gamma * self.planet.TL0 - self.planet.dTL_dchi * self.planet.chi0 / self.fC(self.planet.r_OC / self.planet.L_rho, 0.) * self.planet.r_IC / self.planet.L_rho)    
        print("Secular cooling", self.C,"J")
#         self.C, i = integrate.quad(self.Pc, 0, self.planet.r_IC)
#         print("Secular cooling", self.C,"J")

        ''' Gravitational energy '''
        self.G = 8 * np.pi**2. / 15. * self.planet.chi0 * self.planet.GC * self.planet.rho_c**2. * self.planet.beta * self.planet.r_IC**3. * self.planet.r_OC**5. / self.planet.L_rho**3. \
            / self.fC(self.planet.r_OC/self.planet.L_rho,0)*(1. - self.planet.r_IC**2 / self.planet.r_OC**2 + 3. * self.planet.r_IC**2. / 5. / self.planet.L_rho**2. \
                - 13. * self.planet.r_OC**2. / 14. / self.planet.L_rho**2. + 5./18. * self.planet.r_IC**3. * self.planet.L_rho**2. /self.planet.r_OC**5.)
        print("Gravitational energy", self.G,"J")

        ''' Total energy '''
        self.E_tot = self.L + self.C + self.G
        print("Total energy", self.E_tot,"J")


    def profiles(self, N=50):

        r_IC_vec = np.linspace(self.planet.r_IC_0, 1e8, N) # choose arbitrary value for final core size

        L_H = np.zeros(N)
        S_C = np.zeros(N)
        G_H = np.zeros(N)
        time = np.zeros(N)
        drIC_dt = np.zeros(N)
        PL = np.zeros(N)
        Pc = np.zeros(N)
        Px = np.zeros(N)

        for i, radius_ic in enumerate(r_IC_vec):
            L_H[i] = self.LH(radius_ic)
            S_C[i] = self.SC(radius_ic)
            G_H[i] = self.GH(radius_ic)
            PL[i]  = self.PL(radius_ic)
            #Pc[i]  = self.PC(radius_ic)
            Px[i]  = self.Px(radius_ic)
            #drIC_dt[i] = Q_cmb / (self.PL[radius_ic]+ self.PC[radius_ic] + self.Px[radius_ic])

        E_tot = L_H + S_C + G_H
        time = E_tot / self.planet.Q_CMB/(np.pi*1e7*1e9) 
                 
        plt.figure(1)
        label = ['Latent heating','Secular cooling','Gravitational heating','Total energy']
        plt.plot(time, L_H,label=label[0])
        plt.plot(time, S_C,label=label[1]) 
        plt.plot(time, G_H, label=label[2])
        plt.plot(time, E_tot,label=label[3])
        plt.legend()
        plt.xlabel('Time (Gyr)')
        plt.ylabel('Energy (J)')

        L_H= np.zeros((N,4),dtype=np.float32)
        S_C = np.zeros((N,4),dtype=np.float32)
        G_H = np.zeros((N,4),dtype=np.float32)
        time = np.zeros((N,4),dtype=np.float32)

        for i in range(len(r_IC_vec)):
            L_H[i] = self.LH(r_IC_vec[i])
            S_C[i] = self.SC(r_IC_vec[i])
            G_H[i] = self.GH(r_IC_vec[i])

        E_tot = L_H + S_C + G_H
        time = (L_H + S_C + G_H) / Q_CMB/(np.pi*1e7*1e9)

        plt.figure(3)
        plt.plot(time, r_IC_vec*1e-3)
        plt.xlabel('Time (Gyr)')
        plt.ylabel('Inner core size (km)')
        plt.show()

In [89]:
Mp = 1.2
XFe = 50
FeM = 0.00

class Exo(Rocky_Planet):
    
    GC = 6.67384e-11 # m3/kg/s2

    def parameters(self):
        self.read_parameters("M_ {:.1f}_Fe_{:.0f}.0000_FeM_{:2.0f}.0000.yaml".format(Mp, XFe, FeM))
        qcmb_ev = pd.read_csv("qc_T_M{:02d}_Fe{:02d}_FeM{:02d}.txt".format(int(10*Mp),int(XFe)+5, int(FeM)), sep=" ", skiprows=1, header=None)
        qcmb_ev.columns = ["time", "qcmb", "Tcmb"]
                        

In [90]:
Exo().evolution()

Latent heat 1.1936076334064521e+29 J
Secular cooling 2.8040845430515134e+29 J
Gravitational energy 9.741978597895493e+28 J
Total energy 4.971890036247515e+29 J


TypeError: can't convert complex to float